In [1]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [44]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 2,
                                   22: 2, 23: 2, 24: 2, 25: 5, 26: 2,
                                   27: 8, 28: 4, 29: 21, 30: 12,
                                   31: 16, 32: 23, 33: 13, 34: 24,
                                   35: 21, 36: 26, 37: 22, 38: 25,
                                   39: 34, 40: 26, 41: 31, 42: 32,
                                   43: 32, 44: 30, 45: 34, 46: 30,
                                   47: 30, 48: 36, 49: 33, 50: 35,
                                   51: 34, 52: 28, 53: 42, 54: 34,
                                   55: 45, 56: 30, 57: 40, 58: 27,
                                   59: 33, 60: 32, 61: 17, 62: 18,
                                   63: 11})

In [45]:
chi_last_day = max(chi_death_dict.keys())

In [46]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [47]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [48]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-04-28,33
2020-04-29,32
2020-04-30,17
2020-05-01,18
2020-05-02,11


In [49]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')

### Illinois
Data source for Illinois:

http://www.dph.illinois.gov/covid19/covid19-statistics

`il_cum_death_list` is the list of cumulative deaths on each day, starting on March 16. Again, save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [23]:
il_cum_death_list = [0, 1, 3, 4, 5, 6, 9, 12, 16, 19, 26, 34, 47,
                     65, 72, 99, 141, 157, 210, 243, 274, 307, 380,
                     462, 528, 596, 677, 720, 794, 868, 948, 1072,
                     1134, 1259, 1290, 1349, 1468, 1565, 1688,
                     1795, 1874, 1933, 1983, 2125, 2215, 2355]

In [24]:
# New deaths from March 17
il_death_list = [now - prev 
                 for now, prev in zip(il_cum_death_list[1:],
                                      il_cum_death_list[:-1])]

In [25]:
il_death_dict = defaultdict(int, {index + 17: diff
                                  for index, diff in enumerate(il_death_list)})

In [26]:
il_num_days = len(il_death_list)

In [27]:
num_days = len(il_death_list)

In [28]:
il_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + 16 + num_days)]
il_deaths =[il_death_dict[day - 28] for day in range(29 + 16 + num_days)]

In [29]:
il_death_df = pd.DataFrame(il_deaths, index=il_index, columns=['deaths'])

In [30]:
# Inspect that last few days
il_death_df.tail()

,deaths
2020-04-26,59
2020-04-27,50
2020-04-28,142
2020-04-29,90
2020-04-30,140


In [31]:
il_death_df.to_pickle('../data/il_deaths.pickle')